In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from misc import Timer
from world import BoxWorld

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


# Define the planning world

Define the world with some obstacles

In [3]:
world = BoxWorld([[0, 10], [0, 10]])
world.add_box(2, 2, 6, 6)
world.add_box(1, 6, 4, 3)
world.add_box(4, 1, 5, 4)

plt.figure(10, clear=True)
world.draw()
plt.xlabel('x')
plt.ylabel('y')
plt.axis([world.xmin, world.xmax, world.ymin, world.ymax]);

# Implementation of RRT

Implementation of the RRT planning algorithm

In [4]:
def rrt_particle(start, goal, w, opts):
    """RRT planner for particle moving in a 2D world
    
    Input arguments:
        start - initial state
        goal - desired goal state
        world - description of the map of the world
                using an object from the class BoxWorld
        opts - structure with options for the RRT*

    Output arguments:
        goal_idx - index of the node closest to the desired goal state
        nodes - 2 x N matrix with each column representing a state j
                in the tree
        parents - 1 x N vector with the node number for the parent of node j 
                  at element j in the vector (node number counted as column
                  in the matrix nodes)
        Tplan - the time taken for computing the plan        
    """
    rg = np.random.default_rng()  # Get the random number generator
    def SampleFree():
        """Sample a state x in the free state space"""

        if rg.uniform(0, 1, 1) < opts['beta']:
            return np.array(goal)
        else:
            foundRandom = False
            while not foundRandom:
                x = (rg.uniform(0, 1, 2) * [w.xmax - w.xmin, w.ymax - w.ymin] + 
                     [w.xmin, w.ymin])
                if w.ObstacleFree(x[:, None]):
                    foundRandom = True
            return x

#    def Nearest(x):
#        """Find index of state nearest to x in the matrix nodes"""        
#        idx = np.argmin(np.sum((nodes - x[:, None])**2, axis=0))
#        return idx
    
    
    def Nearest(x):
            N=start
            a=Distance(N[0],N[1],x[0],x[1])
            for i in nodes:
                    if Distance(i[0],i[1],x[0],x[1])<= a :
                            a=Distance(i[0],i[1],x[0],x[1])
                            N=i
            return N   
    
    def Distance(x1,y1,x2,y2):  
        dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
        return dist
    
    def Steer(x1, x2):
        """Steer from x1 towards x2 with step size opts['delta']
        
        If the distance to x2 is less than opts['delta'], return
        state x2.
        """        
        
        dx = np.linalg.norm(x2 - x1)
        if dx < opts['delta']:
            x_new = x2
        else:
            x_new = x1 + opts['delta'] * (x2 - x1) / dx
        return x_new

    
    # Start time measurement and define variables for nodes and parents
    T = Timer()
    T.tic()
#    nodes = np.array(start).reshape((-1, 1))  # Make numpy column vector
    parents = [0]

    my_list = []
    my_list.append(start)
    plt.plot(start[0], start[1] , marker='o', markerfacecolor='blue', markersize=7)
    plt.plot(goal[0], goal[1] , marker='o', markerfacecolor='green', markersize=7)
    for i in range (opts['K']):
            
                    nodes = np.array(my_list)
                    x=SampleFree()
                    N=Nearest(x)
                    if w.ObstacleFree(Steer(N, x)[:, None]):
                            S=Steer(N, x)
                            my_list2 = [arr.tolist() for arr in my_list]
                            N=N.tolist()
                            r=my_list2.index(N)
                            parents.append(r)
                            my_list.append(S)
                            nodes = np.array(my_list)
                            if Distance(S[0],S[1],goal[0], goal[1]) <=opts['eps'] or Distance(S[0],S[1],goal[0], goal[1])==0:
                                   idx_goal=len(my_list)-1
                                   break

                            x1 = [S[0], N[0]] 
                            y1 = [S[1], N[1]]
                            plt.plot(x1, y1, color='green')
    path=[]
    a=idx_goal
    pathfound=False
    while not pathfound==True:
        path.append(parents[a])
        a=parents[a]
        if a==0:
            pathfound=True
    path.reverse()
    length=0
    for i in range(len(path)-1) :
        n=nodes[path[i]]
        m=nodes[path[i+1]]
        x1 = [n[0], m[0]] 
        y1 = [n[1], m[1]]
        plt.plot(x1, y1, color='blue')
        length=length+Distance(n[0],n[1],m[0],m[1])
    Tplan = T.toc()
    
#    idx_goal = np.argmin(np.sum((nodes - np.array(goal).reshape((-1, 1)))**2, axis=0))
    
    return idx_goal, nodes, parents, Tplan, path, length

Run the planner

In [5]:
start = np.array([1, 1])
goal = np.array([9, 9])

opts = {'beta': 0.05,  # Probability for selecting goal state as target state
        'delta': 0.1,  # Step size
        'eps': -0.01,  # Threshold for stopping the search
        'K': 5000}     # Maximum number of iterations

print('Planning ...')
idx_goal, nodes, parents, T, path, length = rrt_particle(start, goal, world, opts)
print('Finished in {:.2f} sek'.format(T))
print('The total number of nodes in the tree is: ',len(nodes)) 
print('The number of nodes along the path is: ',len(path)) 
print('The length of the path is: ',length) 
print('Finished in {:.2f} sek'.format(T))

Planning ...
Finished in 3.23 sek
The total number of nodes in the tree is:  886
The number of nodes along the path is:  193
The length of the path is:  19.200000000000006
Finished in 3.23 sek


# Plots and analysis

In [6]:
plt.show()